In [1]:
import configparser
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import time
import re
import hashlib

import sqlite3
import csv
import os



In [2]:
# get tokens
config = configparser.ConfigParser()
config.read('configfile.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
# authenticate
auth = tweepy.OAuth2AppHandler(api_key, api_key_secret)

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
def get_tweets_dataframe(working_df, tweets, hashtag):
    #working_df=working_df.drop_duplicates()
    print(f'working on {hashtag}')
    index=len(working_df)
    for tweet in tweets: 
        working_df.loc[index,'tweet_id']=tweet.id
        working_df.loc[index,'created_at']=tweet.created_at
        working_df.loc[index,'user']=tweet.user.screen_name
        working_df.loc[index,'full_text']=tweet.full_text
        working_df.loc[index,'favorite_count']=tweet.favorite_count
        working_df.loc[index,'retweet_count']=tweet.retweet_count
        working_df.loc[index,'hashtags']=hashtag
        #print(working_df.loc[index,'created_at'])
        #working_df=working_df.drop_duplicates()
        index+=1
        working_df.to_csv(f'source/{hashtag}.csv', index=False)
        #time.sleep(1)
    working_df=working_df.drop_duplicates()
    working_df.to_csv(f'source/{hashtag}.csv', index=False)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)
          
    return working_df

In [5]:
def get_tweets_from_general_tags(working_df, hashtag):
    #load and merge the general_tags
    print(f'working on {hashtag}')
    general_tags = ['#SaveLockwoodandCo', 'Lockwood and Co', 'Lockwood']
    general_tags_df=pd.DataFrame()
    for tag in general_tags:
        try:
            general_tags_df=pd.concat([general_tags_df, pd.read_csv(f'source/{tag}.csv')], ignore_index=True)
        except:
            pass
    general_tags_df =general_tags_df.drop_duplicates()
    print(f'total generaltags: {len(general_tags_df)}')
    #general_tags_df['full_text'] = general_tags_df['full_text'].apply(lambda x: x.replace('\n',' ').replace(':', ' '))
    #Get the rows in the working_df that have the hashtag inside the full_text column
    hashtag_df = general_tags_df[general_tags_df['full_text'].apply(lambda x: hashtag.lower() in x.lower())]
    #hashtag_df = hashtag_df.reset_index(drop=True)
    hashtag_df['hashtags'] = hashtag
    working_df=pd.concat([working_df, hashtag_df], ignore_index=True)
    working_df=working_df.drop_duplicates()
    working_df['full_text'] = working_df['full_text'].apply(lambda x: x.replace('\n',' ').replace(':', ' '))
    working_df.to_csv(f'source/{hashtag}.csv', index=False)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)
  
    return working_df

In [6]:
# Create a hash function to generate anonymized values
def anonymize(value):
    # Convert the value to a string and hash it using SHA256 algorithm
    hashed_value = hashlib.sha256(str(value).encode()).hexdigest()
    # Take the first 8 characters of the hash as the anonymized value
    anonymized_value = hashed_value[:8]
    return anonymized_value

In [7]:
hashtagsweek1 = ['#FridayNightatPortlandRow', '#HauntedWatchParty', '#WatchPartyatPortlandRow', '#HauntedbyaType3','#TogetherForLockwoodandCo','#PrimeForLockwoodandCo','#BringBackLockwoodandCo']
hashtagsweek2 = ['#GhostHuntersWatchParty', '#DisneyForLockwoodandCo', '#BBCforLockwoodandCo', '#AppleTVforLockwoodandCo', '#PrimeForLockwoodandCo', '#JustRecklessEnough']
hashtagsweek3 = ['#LockwoodGhostAuditions', '#ParamountForLockwoodandCo', '#ScullandCo','#RapiersReady', '#CaringforCarlyle', '#DEPRACisOnTheWay', '#BunsForBunchurch']
hashtagsweek4 = ['#CompleteFictionAppreciation', '#DisneySaveLockwood', '#ArtistryofLockwoodandCo', '#ParamountSaveLockwood', '#GhostStrike', '#LockwoodParallelFandoms', '#JustRecklessEnough']
hashtagsweek5 = ['#StroudsAppreciation', '#VoteLockwoodforNFA', '#PrimeSaveLockwood', '#ScreamingStaircase', '#DEPRACrollcall', '#LivingforLockwood', '#RapiersReady']
hashtagsweek6 = ['#LockNationAppreciation', '#LockNationArtistsandGiftsDay', '#LockNationEditorsDay', '#LockNationFicWritersDay', '#LockNationComediansDay', '#GhostLockAwards']
hashtagsweek7 = ['VoteLocwoodforNFA', '#LockwoodCastAppreciation', '#YouMeAndHerons', '#FishAndKipps', '#LockwoodDnDCampaign', '#TheIronTrio', '#VirtualThinkingCloth']
hashtagsall = hashtagsweek1+hashtagsweek2+hashtagsweek3+hashtagsweek4 + hashtagsweek5 + hashtagsweek6 + hashtagsweek7
hashtagsall = list(set(hashtagsall))

In [35]:

phrase = 'Lockwood and Co'
try:
    working_df = pd.read_csv(f'source/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=phrase, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)



14285
working on Lockwood and Co


Rate limit reached. Sleeping for: 89


KeyboardInterrupt: 

In [36]:

hashtag = '#SaveLockwoodandCo'
try:
    working_df = pd.read_csv(f'source/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)

10592
working on Lockwood and Co


KeyboardInterrupt: 

In [ ]:
phrase = 'Lockwood'
try:
    working_df = pd.read_csv(f'source/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=phrase, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)

working on Lockwood
created_at
2023-06-26 00:00:00+00:00    149
Freq: D, Name: hashtags, dtype: int64
149


In [10]:
hashtags = ['#LockNationArtistsandGiftsDay']
for hashtag in hashtagsall:
    try:
        working_df=pd.read_csv(f'source/{hashtag}.csv')
        #working_df = pd.read_csv('source/#SkullandCo.csv')
        print(len(working_df))
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
    tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
    working_df = get_tweets_dataframe(working_df, tweets, hashtag)
    working_df=working_df.drop_duplicates()
    print(len(working_df))
    time.sleep(10)

5866
working on #DisneyForLockwoodandCo


/tmp/ipykernel_822/3429170079.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)


created_at
2023-05-22 00:00:00+00:00      35
2023-05-23 00:00:00+00:00    4179
2023-05-24 00:00:00+00:00    1275
2023-05-25 00:00:00+00:00     231
2023-05-26 00:00:00+00:00      45
2023-05-27 00:00:00+00:00       6
2023-05-28 00:00:00+00:00      13
2023-05-29 00:00:00+00:00      13
2023-05-30 00:00:00+00:00       2
2023-05-31 00:00:00+00:00       7
2023-06-01 00:00:00+00:00       2
2023-06-02 00:00:00+00:00       4
2023-06-03 00:00:00+00:00       0
2023-06-04 00:00:00+00:00       0
2023-06-05 00:00:00+00:00       2
2023-06-06 00:00:00+00:00      11
2023-06-07 00:00:00+00:00       0
2023-06-08 00:00:00+00:00       0
2023-06-09 00:00:00+00:00       0
2023-06-10 00:00:00+00:00       1
2023-06-11 00:00:00+00:00       3
2023-06-12 00:00:00+00:00       0
2023-06-13 00:00:00+00:00       0
2023-06-14 00:00:00+00:00       0
2023-06-15 00:00:00+00:00       0
2023-06-16 00:00:00+00:00       1
2023-06-17 00:00:00+00:00       4
2023-06-18 00:00:00+00:00       2
2023-06-19 00:00:00+00:00       0
202

Rate limit reached. Sleeping for: 389


created_at
2023-06-26 00:00:00+00:00      25
2023-06-27 00:00:00+00:00    6121
2023-06-28 00:00:00+00:00    1566
2023-06-29 00:00:00+00:00     112
2023-06-30 00:00:00+00:00      12
2023-07-01 00:00:00+00:00       1
2023-07-02 00:00:00+00:00       1
Freq: D, Name: hashtags, dtype: int64
4741
6289
working on #CaringforCarlyle
created_at
2023-05-28 00:00:00+00:00       1
2023-05-29 00:00:00+00:00       1
2023-05-30 00:00:00+00:00       0
2023-05-31 00:00:00+00:00       0
2023-06-01 00:00:00+00:00       9
2023-06-02 00:00:00+00:00    3090
2023-06-03 00:00:00+00:00    1086
2023-06-04 00:00:00+00:00      50
2023-06-05 00:00:00+00:00       3
2023-06-06 00:00:00+00:00       0
2023-06-07 00:00:00+00:00       2
2023-06-08 00:00:00+00:00       6
2023-06-09 00:00:00+00:00       1
2023-06-10 00:00:00+00:00       0
2023-06-11 00:00:00+00:00       3
2023-06-12 00:00:00+00:00       1
2023-06-13 00:00:00+00:00       3
2023-06-14 00:00:00+00:00       4
2023-06-15 00:00:00+00:00       1
Freq: D, Name: ha

Rate limit reached. Sleeping for: 402


created_at
2023-06-29 00:00:00+00:00      21
2023-06-30 00:00:00+00:00    2904
2023-07-01 00:00:00+00:00     339
2023-07-02 00:00:00+00:00      17
2023-07-03 00:00:00+00:00       3
Freq: D, Name: hashtags, dtype: int64
3232
5380
working on #BBCforLockwoodandCo
created_at
2023-05-22 00:00:00+00:00      27
2023-05-23 00:00:00+00:00       1
2023-05-24 00:00:00+00:00    3881
2023-05-25 00:00:00+00:00    1297
2023-05-26 00:00:00+00:00     108
2023-05-27 00:00:00+00:00      13
2023-05-28 00:00:00+00:00      11
2023-05-29 00:00:00+00:00       8
2023-05-30 00:00:00+00:00       4
2023-05-31 00:00:00+00:00       4
2023-06-01 00:00:00+00:00       0
2023-06-02 00:00:00+00:00       0
2023-06-03 00:00:00+00:00       3
2023-06-04 00:00:00+00:00       1
2023-06-05 00:00:00+00:00       2
2023-06-06 00:00:00+00:00       0
2023-06-07 00:00:00+00:00       0
2023-06-08 00:00:00+00:00       2
2023-06-09 00:00:00+00:00       2
2023-06-10 00:00:00+00:00       0
2023-06-11 00:00:00+00:00       0
2023-06-12 00:

Rate limit reached. Sleeping for: 546


created_at
2023-06-03 00:00:00+00:00       8
2023-06-04 00:00:00+00:00       6
2023-06-05 00:00:00+00:00       0
2023-06-06 00:00:00+00:00       0
2023-06-07 00:00:00+00:00       0
2023-06-08 00:00:00+00:00       0
2023-06-09 00:00:00+00:00       0
2023-06-10 00:00:00+00:00       0
2023-06-11 00:00:00+00:00       0
2023-06-12 00:00:00+00:00       0
2023-06-13 00:00:00+00:00       0
2023-06-14 00:00:00+00:00       0
2023-06-15 00:00:00+00:00       0
2023-06-16 00:00:00+00:00       0
2023-06-17 00:00:00+00:00       0
2023-06-18 00:00:00+00:00       0
2023-06-19 00:00:00+00:00       0
2023-06-20 00:00:00+00:00       0
2023-06-21 00:00:00+00:00       0
2023-06-22 00:00:00+00:00       0
2023-06-23 00:00:00+00:00       0
2023-06-24 00:00:00+00:00       0
2023-06-25 00:00:00+00:00       0
2023-06-26 00:00:00+00:00       0
2023-06-27 00:00:00+00:00       0
2023-06-28 00:00:00+00:00      46
2023-06-29 00:00:00+00:00    5111
2023-06-30 00:00:00+00:00     731
2023-07-01 00:00:00+00:00      10
Fre

In [11]:
hashtags = ['#SkullandCo']
for hashtag in hashtagsall:
    try:
        
        working_df=pd.read_csv(f'source/{hashtag}.csv')
        print('try')
        working_df = get_tweets_from_general_tags(working_df, hashtag)
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
        print('except')
        working_df = get_tweets_from_general_tags(working_df, hashtag)
        

try
working on #DisneyForLockwoodandCo
total generaltags: 63930


/tmp/ipykernel_822/3339171900.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hashtag_df['hashtags'] = hashtag


created_at
2023-05-22 00:00:00+00:00      35
2023-05-23 00:00:00+00:00    4179
2023-05-24 00:00:00+00:00    1275
2023-05-25 00:00:00+00:00     231
2023-05-26 00:00:00+00:00      45
2023-05-27 00:00:00+00:00       6
2023-05-28 00:00:00+00:00      17
2023-05-29 00:00:00+00:00      15
2023-05-30 00:00:00+00:00       3
2023-05-31 00:00:00+00:00      11
2023-06-01 00:00:00+00:00       4
2023-06-02 00:00:00+00:00       6
2023-06-03 00:00:00+00:00       0
2023-06-04 00:00:00+00:00       0
2023-06-05 00:00:00+00:00       3
2023-06-06 00:00:00+00:00      12
2023-06-07 00:00:00+00:00       0
2023-06-08 00:00:00+00:00       0
2023-06-09 00:00:00+00:00       0
2023-06-10 00:00:00+00:00       2
2023-06-11 00:00:00+00:00       5
2023-06-12 00:00:00+00:00       0
2023-06-13 00:00:00+00:00       0
2023-06-14 00:00:00+00:00       0
2023-06-15 00:00:00+00:00       0
2023-06-16 00:00:00+00:00       1
2023-06-17 00:00:00+00:00       4
2023-06-18 00:00:00+00:00       2
2023-06-19 00:00:00+00:00       0
202

In [12]:
working_df=pd.DataFrame()
for hashtag in hashtagsall:
    try:
        working_df=pd.concat([working_df, pd.read_csv(f'source/{hashtag}.csv')], ignore_index=True)
    except:
        pass
len(working_df)

253496

In [13]:
working_df['full_text'] = working_df['full_text'].str.replace(r'"', '').replace(r'\n', ' ').replace(':', ' ').replace(',', '').replace('!', '')

In [14]:
# Create a new column 'retweet' with default value False
working_df['retweet'] = False

# Check if 'full_text' starts with 'RT ' and set 'retweet' column accordingly
working_df.loc[working_df['full_text'].str.startswith('RT '), 'retweet'] = True

#Remove the 'RT ' in the full_text column
working_df['full_text'] = working_df['full_text'].str.replace('RT ', '')

working_df['retweet'].value_counts()

False    145762
True     107734
Name: retweet, dtype: int64

In [15]:
def get_week_label(week_number):
    if week_number == 20:
        return 'week1'
    elif week_number == 21:
        return 'week2'
    elif week_number == 22:
        return 'week3'
    elif week_number == 23:
        return 'week4'
    elif week_number == 24:
        return 'week5'
    elif week_number == 25:
        return 'week6'
    elif week_number == 26:
        return 'week7'
    else:
        return 'week1'


In [16]:
working_df['created_date'] = pd.to_datetime(working_df['created_at'], utc=True)
working_df['week'] = working_df['created_date'].dt.isocalendar().week
working_df['week'] = working_df['week'].apply(get_week_label)
working_df['week'].value_counts()

week4    47107
week1    45736
week3    45268
week2    36116
week5    30084
week6    25468
week7    23717
Name: week, dtype: int64

In [17]:
#Create an anonymized tweet_id
#working_df['unique_id'] = str(working_df['tweet_id']) + working_df['created_at'] + working_df['user'] + str(working_df['retweet'])
working_df['unique_id'] = working_df.apply(lambda row: str(row['tweet_id']) + row['created_at'] + row['user'] + str(row['retweet']), axis=1)
working_df['tweet_id'] = working_df['unique_id'].apply(anonymize)

In [18]:
hashtags_df = working_df[['tweet_id','hashtags']]
hashtags_df.to_csv('output/hashtags.csv', index=False)
working_df.value_counts('hashtags')

hashtags
#JustRecklessEnough              18890
#PrimeForLockwoodandCo           15253
#RapiersReady                    13198
#ParamountForLockwoodandCo        9964
#TogetherForLockwoodandCo         9415
#BunsForBunchurch                 9282
#ParamountSaveLockwood            9197
#CompleteFictionAppreciation      8823
#VoteLockwoodforNFA               8184
#BringBackLockwoodandCo           7882
#LockwoodParallelFandoms          7761
#ScreamingStaircase               6630
#LockNationEditorsDay             6619
#LockwoodGhostAuditions           6465
#GhostStrike                      6348
#CaringforCarlyle                 6289
#DisneyForLockwoodandCo           5865
#HauntedbyaType3                  5811
#GhostLockAwards                  5432
#DEPRACisOnTheWay                 5426
#BBCforLockwoodandCo              5380
#WatchPartyatPortlandRow          5255
#FridayNightatPortlandRow         5218
#DisneySaveLockwood               5023
#LockNationAppreciation           4950
#HauntedWatchPar

In [19]:
# Get the tagged users from the full_text
working_df=working_df.drop_duplicates('tweet_id')
filter_out = ['@AppleSupport ','@AppleTV ','@AppleTVI ','@AppleTVPlus ','@AppleTVUK ','@BBC ', '@BBCOne ', '@BBCPlayer ','@BBCSounds ', '@BBCR1 ',
            '@BBCSounds ','@BBCiPlayer ','@BBCone ','@D ','@Di ','@Dis ','@Disn ', '@Disne ', '@Dinsey ','@DisneyChannel ','@DisneyHyperion ','@DisneyIT ', '@DisneyPlus ', '@DisneyChannel ',
            '@DisneyHyperion ','@DisneyPlusUK', '@DisneyPlus', '@HBO_UK ','@HBO ', '@hulu ','@NETFLIX ', '@netflix ', '@NETFLIXUK ', '@Netflix ', '@NetflixUK ','@ParamountPlus ', '@ParamaountUK ', '@ParamountPlusUK '
            '@Prime ', '@PrimeUK ', '@PrimeVideo ', '@PrimeVideo','@primevideouk ']
for filter in filter_out:
    working_df['full_text'] = working_df['full_text'].str.replace(filter, '')
working_df['tagged_users'] = working_df['full_text'].apply(lambda x: re.findall(r'@(\w+)', x))
# Remove the tagged users from the full_text
working_df['full_text'] = working_df['full_text'].str.replace(r'@(\w+)', '')
list_of_tagged_users = []
for x in range(len(working_df)):
    try:
        list_of_tagged_users += working_df['tagged_users'][x]
    except:
        pass



/tmp/ipykernel_822/3040538264.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  working_df['full_text'] = working_df['full_text'].str.replace(r'@(\w+)', '')


In [20]:
#Create a table for network analysis
na_df = working_df[['tweet_id','user', 'retweet', 'tagged_users']]

In [21]:
#Create a function that would create a dataframe that shows the interaction
def get_interaction(df):
    func_df = pd.DataFrame(columns = ['tweet_id', 'from_', 'to_'])
    filter_out = ['2','A','App','AppleSupport','AppleTV','AppleTVI','AppleTVPlus','AppleTVUK','BB','BBC','BBCO','BBC','BBCOn', 'BBCOne', 'BBCPlayer','BBCSounds', 'BBCR1',
                  'BBCSounds','BBCiPlayer','BBCone','D','Di','Dis','Disn', 'Disne', 'Dinsey',' DisneyChannel','DisneyHyperion','DisneyIT', 'DisneyP', 'DisneyChannel',
                  'DisneyHyperion','DisneyPlusUK', 'DisneyPlus', 'HBO', 'HBO_UK','NETFLIX', 'netflix', 'NETFLIXUK', 'Netflix', 'NetflixUK','ParamountPlus', 'ParamaountUK', 'ParamountPlusUK'
                  'Prime', 'PrimeUK', 'PrimeVideo', 'primevideouk']
    for x in range(len(df)):
        if df['retweet'][0]=='True':
            if df['tagged_users'].iloc[x][0] not in filter_out:
                new_row = {'tweet_id':df['tweet_id'].iloc[x], 'from_':df['tagged_users'].iloc[x][0], 'to_': df['user'].iloc[x]}
                func_df = func_df.append(new_row, ignore_index = True)
            else:
                print(df['tagged_users'].iloc[x][0])
        else:
            for user in df['tagged_users'].iloc[x]:
                if user not in filter_out:
                    new_row = {'tweet_id':df['tweet_id'].iloc[x], 'to_':user, 'from_': df['user'].iloc[x]}
                    func_df = func_df.append(new_row, ignore_index = True)
                else:
                    print(user)
    return func_df.drop_duplicates()


In [22]:
na_interac_df =get_interaction(na_df)
filter_out = ['2','A','App','AppleSupport','AppleTV','AppleTVI','AppleTVPlus','AppleTVUK','BB','BBC','BBCO','BBC','BBCOn', 'BBCOne', 'BBCPlayer','BBCSounds', 'BBCR1',
                  'BBCSounds','BBCiPlayer','BBCone','D','Di','Dis','Disn', 'Disne', 'Dinsey',' DisneyChannel','DisneyHyperion','DisneyIT', 'DisneyP', 'DisneyChannel',
                  'DisneyHyperion','DisneyPlusUK', 'DisneyPlus', 'HBO', 'HBO_UK','NETFLIX', 'netflix', 'NETFLIXUK', 'Netflix', 'NetflixUK','ParamountPlus', 'ParamaountUK', 'ParamountPlusUK'
                  'Prime', 'PrimeUK', 'PrimeVideo', 'primevideouk']
na_interac_df = na_interac_df[~na_interac_df['from_'].isin(filter_out)]
na_interac_df = na_interac_df[~na_interac_df['to_'].isin(filter_out)]                            
print(na_interac_df[na_interac_df['from_'].isin(filter_out)])
print(na_interac_df[na_interac_df['to_'].isin(filter_out)])

primevideouk
Dis
Disn
DisneyP
Dis
D
Disn
Disn
Dis
Disn
Disn
D
Dis
Dis
D
Disn
Dis
D
DisneyP
Dis
Dis
Di
D
D
Disn
DisneyP
D
Disn
Disn
D
Disn
D
Disn
Disn
Disn
DisneyP
Disn
Disn
D
Disn
D
D
D
D
D
netflix
Disn
D
Disn
D
NetflixUK
Disn
Disn
Disn
D
D
D
D
D
Disn
D
Disn
Disn
Disn
D
D
D
Disn
D
D
D
D
Disn
D
D
D
Disn
D
D
D
Disn
Disn
Disn
D
Disn
Disn
Disn
Disn
Disn
Disn
D
D
Disn
D
Disn
Disn
D
Disn
Disn
Disn
Disn
Disn
D
D
netflix
BBCOne
primevideouk
primevideouk
primevideouk
primevideouk
primevideouk
primevideouk
primevideouk
primevideouk
primevideouk
primevideouk
primevideouk
NetflixUK
Dis
DisneyP
netflix
netflix
netflix
netflix
netflix
netflix
ParamountPlus
DisneyP
DisneyP
netflix
netflix
netflix
netflix
netflix
netflix
primevideouk
NetflixUK
netflix
netflix
netflix
NetflixUK
netflix
D
netflix
netflix
netflix
NetflixUK
NetflixUK
NetflixUK
netflix
netflix
netflix
NetflixUK
netflix
primevideouk
netflix
netflix
netflix
netflix
primevideouk
netflix
netflix
netflix
netflix
D
netflix
netflix
Disne
netflix


In [23]:
#get a unique user dataframe
list_of_user = na_interac_df['from_'].append(na_interac_df['to_'], ignore_index=True)
list_of_user = list(set(list_of_user))
user_df = pd.DataFrame(list_of_user, columns=['username'])
user_df['user'] = user_df['username'].apply(anonymize)
user_df=user_df.sort_values('username').reset_index()
user_df.to_csv('output/username.csv', index=False)

In [24]:
#anonymise the users from the different dataframes
na_interac_df['from_'] = na_interac_df['from_'].map(user_df.set_index('username')['user'])
na_interac_df['to_'] = na_interac_df['to_'].map(user_df.set_index('username')['user'])
working_df['user'] = working_df['user'].map(user_df.set_index('username')['user'])
na_interac_df.to_csv('output/userinteraction.csv', index=False)

In [25]:
working_df = working_df[['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'retweet', 'week']]
working_df.to_csv('output/tweets.csv', index=False)

In [26]:
set(hashtags_df['tweet_id'])==set(working_df['tweet_id'])

True

In [27]:
# remove existing database if available
if os.path.exists('output/lnctweets.db'):
    os.remove('output/lnctweets.db')
# Creating a database file for the all the output csv file
# Establish a connection to the SQLite Database
conn = sqlite3.connect('output/lnctweets.db')
# Create a cursor object to execute SQL statements:
cursor = conn.cursor()

In [28]:
def create_table(table_name, columns):
    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({', '.join(columns)})"
    cursor.execute(create_table_query)

In [29]:
# Define a function to insert data into the table from a CSV file:
def insert_data(table_name, csv_file):
    with open(csv_file, 'r') as file:
        csv_data = csv.reader(file)
        next(csv_data)  # Skip the header row if necessary
        num_columns = len(next(csv_data))
        insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['?'] * num_columns)})"
        cursor.executemany(insert_query, csv_data)

In [30]:
# Specify the CSV file paths and table names for each file:
csv_files = ['output/hashtags.csv',  'output/tweets.csv', 'output/userinteraction.csv']
table_names = ['hashtags', 'tweets', 'userinteraction']

In [31]:
# Create tables and insert data for each CSV file:
for i, csv_file in enumerate(csv_files):
    table_name = table_names[i]
    with open(csv_file, 'r') as file:
        csv_data = csv.reader(file)
        columns = next(csv_data)
        data_type = ' TEXT'
        columns = [x + data_type for x in columns]
        #print(columns)
        create_table(table_name, columns)
        insert_data(table_name, csv_file)

In [32]:
# Commit the changes and close the connection
conn.commit()
conn.close()

In [33]:
# open the tweets table in the .db file
query = """
    SELECT *
    FROM userinteraction
    """
conn = sqlite3.connect('output/lnctweets.db')
print(pd.read_sql_query(query,conn))
conn.close()

        tweet_id     from_       to_
0       169ea973  facd6e5a  7990bcef
1       2a32f7d5  7200eeb3  63b849cd
2       8e530d74  4a2a5f30  7990bcef
3       b4dc738e  2a64c1ea  7990bcef
4       7ca78770  94be579b  7990bcef
...          ...       ...       ...
186346  95308da5  6a5efece  93fe8986
186347  188a1cfb  6a5efece  a838dedb
186348  188a1cfb  6a5efece  e7ba4fc2
186349  188a1cfb  6a5efece  93fe8986
186350  a675f45d  417ab9da  639397c9

[186351 rows x 3 columns]


In [34]:
len(set(working_df['tweet_id']))

182790